In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_kraken_orderbooks_from_file
from hydradx.model.processing import get_omnipool_data_from_file
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent as arb

from pprint import pprint

import json


Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = True

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [4]:
# arb_file = "arbconfig.txt"
arb_file = "arbconfig2.txt"
with open('config/' + arb_file, 'r') as json_file:
    arb_list = json.load(json_file)
    
for d in arb_list:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])

ob_objs = {}
order_book_asset_list = []
buffer = {}

if get_live_markets:
    for arb_cfg in arb_list:
        tkn_pair = arb_cfg['order_book']
        buffer[(arb_cfg['tkn_ids'],tkn_pair)] = arb_cfg['buffer']
        if tkn_pair not in ob_objs:
            ob_objs[tkn_pair] = get_kraken_orderbook(tkn_pair, archive=True)
            for tkn in tkn_pair:
                if tkn not in order_book_asset_list:
                    order_book_asset_list.append(tkn)
                    
else:
    ob_objs = get_kraken_orderbooks_from_file("data/input/")
    for arb_cfg in arb_list:
        tkn_pair = arb_cfg['order_book']
        buffer[(arb_cfg['tkn_ids'],tkn_pair)] = arb_cfg['buffer']
        for tkn in tkn_pair:
            if tkn not in order_book_asset_list:
                order_book_asset_list.append(tkn)    
                


In [5]:
RPC="wss://rpc.hydradx.cloud"
N = 24
            
if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, N, archive=True)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("data/input/")
    
buffer2 = {((asset_map[k[0][0]], asset_map[k[0][1]]),k[1]): v for k, v in buffer.items()}
pprint(tokens)



{'ASTR': {'LRNA': 9942.639831903161, 'liquidity': 4257078.0188511135},
 'BNC': {'LRNA': 6746.982093386014, 'liquidity': 714414.6769671425},
 'CFG': {'LRNA': 9380.2055093877, 'liquidity': 594474.8844882386},
 'DAI': {'LRNA': 12702.251432922867, 'liquidity': 343412.5919105631},
 'DAI001': {'LRNA': 1407.281258338137, 'liquidity': 37585.53464434876},
 'DOT': {'LRNA': 71345.7052267667, 'liquidity': 392089.8289163709},
 'GLMR': {'LRNA': 5792.299769429669, 'liquidity': 614441.7204258053},
 'HDX': {'LRNA': 21500.1592405937, 'liquidity': 106808509.87427911},
 'INTR': {'LRNA': 8571.197120769379, 'liquidity': 13353025.19535228},
 'USDT': {'LRNA': 83825.69978154043, 'liquidity': 2283455.792091},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41090.31683516522, 'liquidity': 31.62994478},
 'WETH': {'LRNA': 21475.489522148586, 'liquidity': 308.5266368689693},
 'WETH001': {'LRNA': 71124.00046358124, 'liquidity': 1025.6597989911982},
 'ZTG': {'LRNA': 4289.293726236884

In [6]:


cex_fee = 0.0016
# buffer = 0.0010

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0
# asset_fee = 0.0
# cex_fee = 0.0
# buffer = 0.0


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


In [7]:
cex1 = cex.copy()
op_state1 = op_state.copy()
all_swaps1 = arb.get_arb_swaps_simple(op_state1, cex1, buffer=buffer2)

pprint(all_swaps1)
pprint(len(all_swaps1))

cex2 = cex.copy()
op_state2 = op_state.copy()
all_swaps2 = arb.get_arb_swaps(op_state2, cex2, buffer=buffer2)
print("strategy 2")
pprint(all_swaps2)
pprint(len(all_swaps2))

[{'cex': {'amount': 0.13714805106925967,
          'buy_asset': 'DAI',
          'price': 1887.5357600000002,
          'sell_asset': 'ETH',
          'trade': 'sell'},
  'dex': {'amount': 0.13714805106925967,
          'buy_asset': 'WETH001',
          'max_sell': 256.6706178892851,
          'price': 1888.48,
          'sell_asset': 'DAI001',
          'trade': 'buy'}},
 {'cex': {'amount': 0.12934077983665468,
          'buy_asset': 'XBT',
          'price': 35246.71455,
          'sell_asset': 'USDT',
          'trade': 'buy'},
  'dex': {'amount': 0.12934077983665468,
          'buy_asset': 'USDT',
          'min_buy': 4597.864495076239,
          'price': 35229.1,
          'sell_asset': 'iBTC',
          'trade': 'sell'}},
 {'cex': {'amount': 40.0,
          'buy_asset': 'XBT',
          'price': 0.000140439745,
          'sell_asset': 'DOT',
          'trade': 'sell'},
  'dex': {'amount': 40.0,
          'buy_asset': 'DOT',
          'max_sell': 0.005595380902290344,
          'p

In [8]:
state1 = init_state.copy()
state2 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
agent2 = init_agent.copy()
cex1 = cex.copy()
cex2 = cex.copy()

arb.execute_arb(state1, cex1, agent1, all_swaps1)
arb.execute_arb(state2, cex2, agent2, all_swaps2)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'iBTC': 'BTC',
             'XBT': 'BTC',}

profit1 = arb.calculate_profit(init_agent, agent1, asset_map)
profit2 = arb.calculate_profit(init_agent, agent2, asset_map)

asset_map_omnipool = {
    'ETH': 'WETH',
    'XETH': 'WETH',
    'XXBT': 'WBTC',
    'BTC': 'WBTC',
    'ZUSD': 'USDT',
    'USD': 'USDT',
    'USDC': 'USDT',
    'DAI': 'USDT',
    'USDT001': 'USDT',
    'DAI001': 'USDT',
    'WETH001': 'WETH',
    'WBTC001': 'WBTC',
    'iBTC': 'WBTC',
    'XBT': 'WBTC',
}


profit1_omnipool = arb.calculate_profit(init_agent, agent1, asset_map_omnipool)
profit2_omnipool = arb.calculate_profit(init_agent, agent2, asset_map_omnipool)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Omnipool spot")
print(sum([profit1_omnipool[asset] * state1.price(state1, asset, 'USDT') for asset in profit1_omnipool]))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Omnipool spot")
print(sum([profit2_omnipool[asset] * state2.price(state2, asset, 'USDT') for asset in profit2_omnipool]))
            

strategy 1 profit
{'HDX': 0, 'USD': 41.7723388671875, 'BTC': 0.00011241436004638672, 'ETH': 0.0, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0.0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0.0, 'INTR': 0}
strategy 1 profit at Omnipool spot
45.302527692868466
strategy 2 profit
{'HDX': 0, 'USD': 6.313859820365906, 'BTC': 0.0010783672332763672, 'ETH': 0.0, 'DOT': 0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0.0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0.0, 'INTR': 0}
strategy 2 profit at Omnipool spot
40.25959569240577
